<a href="https://colab.research.google.com/github/jcandane/CI_Theory/blob/main/CIround1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Goal of this notebook is to introduce a standalone FCI program in Python.

Import Libraries, set defaults

In [1]:
!pip install pyscf

import numpy as np
from itertools import combinations, permutations
from pyscf import fci, ao2mo, scf, gto

π = np.pi
α = 0.007297352
c = 1.0/α
np.set_printoptions(precision=4, linewidth=200, threshold=2000, suppress=True)

### for proformance reasons, list of determinants should be in np.int8 (1 B, upto 127 dets), np.int16 (2 B, upto 32,767 dets)), np.int32 (4 B, upto 2 billon dets)
CI_dt  = np.int16 ### index that labels CI states, i.e. I, J, K in notes

Import defintions

In [2]:
def ΛMOgetB(Λ, N_mo):
  "Given Λ (i occupied orbitals for each determinant) get B (binary rep.)"

  Binary  = np.zeros((Λ.shape[0], N_mo), dtype=np.int8)
  for I in range(len(Binary)):
      Binary[I, Λ[I,:]] = 1

  return Binary

def givenΛgetB(ΛA, ΛB, N_mo):
  "Given Λ (i occupied orbitals for each determinant) get B (binary rep.)"

  Binary  = np.zeros((ΛA.shape[0], 2, N_mo), dtype=np.int8)
  for I in range(len(Binary)):
      Binary[I, 0, ΛA[I,:]] = 1
      Binary[I, 1, ΛB[I,:]] = 1

  return Binary

def givenBgetΛ(B):
  "Given B (entire MO binary occupation) get Λ (i occupied)"

  numA = len( (B[0,0])[ B[0,0] == 1 ] ) ## count num. of A occupied
  numB = len( (B[0,1])[ B[0,1] == 1 ] ) ## count num. of B occupied

  ΛA = np.zeros((B.shape[0], numA), dtype=np.int8)
  ΛB = np.zeros((B.shape[0], numB), dtype=np.int8)
  for I in range(len(Λ)):
    ΛA[I] = np.where(B == 1)[1]
    ΛB[I] = np.where(B == 1)[1]

  return ΛA, ΛB

def Ext(A, N, numtype=np.int8):
    if A.ndim == 1:
      return np.einsum("I, j -> Ij", A, np.ones( N, dtype=numtype))
    if A.ndim == 2:
      return np.einsum("uI, j -> uIj", A, np.ones( N, dtype=numtype))

def SpinOuterProduct(A, B, stack=False):
  ΛA = np.einsum("Ii, J -> IJi", A, np.ones(B.shape[0], dtype=np.int8)).reshape( (A.shape[0]*B.shape[0], A.shape[1]) )
  ΛB = np.einsum("Ii, J -> JIi", B, np.ones(A.shape[0], dtype=np.int8)).reshape( (A.shape[0]*B.shape[0], B.shape[1]) )
  
  if stack:
    return np.array([ΛA,ΛB])
  else:
    return ΛA, ΛB

def get_SO_matrix(uhf_pyscf, SF=False, H1=None, H2=None):
    """ Given a PySCF uhf object get SO Matrices """

    Ca, Cb = (uhf_pyscf).mo_coeff
    S = (uhf_pyscf.mol).intor("int1e_ovlp")
    eig, v = np.linalg.eigh(S)
    A = (v) @ np.diag(eig**(-0.5)) @ np.linalg.inv(v) 
    H = (uhf_pyscf.mol).intor('int1e_kin') + (uhf_pyscf.mol).intor('int1e_nuc') + (uhf_pyscf.mol).intor("ECPscalar")

    n = Ca.shape[1]
    eri_aa = (ao2mo.general( (uhf_pyscf)._eri , (Ca, Ca, Ca, Ca), compact=False)).reshape((n,n,n,n), order="C")
    eri_aa -= eri_aa.swapaxes(1,3)
    eri_bb = (ao2mo.general( (uhf_pyscf)._eri , (Cb, Cb, Cb, Cb), compact=False)).reshape((n,n,n,n), order="C")
    eri_bb -= eri_bb.swapaxes(1,3)
    eri_ab = (ao2mo.general( (uhf_pyscf)._eri , (Ca, Ca, Cb, Cb), compact=False)).reshape((n,n,n,n), order="C")
    #eri_ba = (1.*eri_ab).swapaxes(0,3).swapaxes(1,2) ## !! caution depends on symmetry
    eri_ba = (ao2mo.general( (uhf_pyscf)._eri , (Cb, Cb, Ca, Ca), compact=False)).reshape((n,n,n,n), order="C")
    H2 = np.stack(( np.stack((eri_aa, eri_ab)), np.stack((eri_ba, eri_bb)) ))

    H1 = np.asarray([np.einsum("AB, Ap, Bq -> pq", H, Ca, Ca), np.einsum("AB, Ap, Bq -> pq", H, Cb, Cb)])

    if SF:
      eri_abab = (ao2mo.general( (uhf_pyscf)._eri , (Ca, Cb, Ca, Cb), compact=False)).reshape((n,n,n,n), order="C")
      eri_abba = (ao2mo.general( (uhf_pyscf)._eri , (Ca, Cb, Cb, Ca), compact=False)).reshape((n,n,n,n), order="C")
      eri_baab = (ao2mo.general( (uhf_pyscf)._eri , (Cb, Ca, Ca, Cb), compact=False)).reshape((n,n,n,n), order="C")
      eri_baba = (ao2mo.general( (uhf_pyscf)._eri , (Cb, Ca, Cb, Ca), compact=False)).reshape((n,n,n,n), order="C")
      H2_SF = np.stack(( np.stack((eri_abab, eri_abba)), np.stack((eri_baab, eri_baba)) ))
      return H1, H2, H2_SF

    else:
      return H1, H2

# Let's now do a PySCF HF Calculation

In [3]:
mol = gto.M(atom='He 0.0 0.0 0.0; H 1.5 0.0 0.0; He 0.0 1.5 0.0; N 0.0, 0.0, 1.5', spin=0, basis="sto-3g")
uhf = scf.UHF(mol)
uhf.kernel()

### information about occupation
O_sp   = np.asarray((uhf).mo_occ, dtype=np.int8)
N_s    = np.einsum("sp -> s", O_sp)

converged SCF energy = -59.3998556537129  <S^2> = 1.5861234e-06  2S+1 = 1.0000032


# Configuration Generation (fCI)

In [4]:
N   = O_sp.shape[1]
Λ_α = np.asarray( list(combinations(  np.arange(0, N, 1, dtype=np.int8)  , N_s[0] ) ) ) 
Λ_β = np.asarray( list(combinations(  np.arange(0, N, 1, dtype=np.int8)  , N_s[1] ) ) ) 
ΛA, ΛB = SpinOuterProduct(Λ_α, Λ_β)
Binary = givenΛgetB(ΛA, ΛB, N)

if len(Binary) < 125:
    CI_dt  = np.int8
if len(Binary) > 32700:
    CI_dt  = np.int32

Determine Determinant Signs

In [5]:
sign  = np.cumsum( Binary, axis=2)
for I in range(len(Binary)):
    sign[I, 0, ΛA[I]] = np.arange(0, N_s[0], 1)
    sign[I, 1, ΛB[I]] = np.arange(0, N_s[1], 1)

Γ_Isp = ( (-1)**(sign) ).astype(np.int8)

Determine Differences

In [6]:
Binary         = givenΛgetB(ΛA, ΛB, N)
Difference     = np.einsum("Isp, J -> IJsp", Binary, np.ones(len(Binary), dtype=np.int8)) - np.einsum("Isp, J -> JIsp", Binary, np.ones(len(Binary), dtype=np.int8))
Sum            = np.einsum("Isp, J -> IJsp", Binary, np.ones(len(Binary), dtype=np.int8)) + np.einsum("Isp, J -> JIsp", Binary, np.ones(len(Binary), dtype=np.int8))
SpinDifference = np.einsum("IJsp -> IJs", np.abs(Difference))//2

# Let's now get SO Matrix Elements

In [7]:
H1, H2 = get_SO_matrix(uhf)

# Let's now get CI Matrix Elements

Slater-Condon Rule 0 & 3

In [8]:
AB = Ext(ΛA, N_s[1])
BA = Ext(ΛB, N_s[0])
AA = Ext(ΛA, N_s[0])
BB = Ext(ΛB, N_s[1])

Rule0  = np.einsum("Ii  -> I", H1[0, ΛA, ΛA]) 
Rule0 += np.einsum("Ii  -> I", H1[1, ΛB, ΛB])
Rule0 += np.einsum("Iij -> I", H2[0, 0, AA, AA, AA.swapaxes(1,2), AA.swapaxes(1,2)])/2
Rule0 += np.einsum("Iij -> I", H2[1, 0, BA, BA, AB.swapaxes(1,2), AB.swapaxes(1,2)])/2
Rule0 += np.einsum("Iij -> I", H2[0, 1, AB, AB, BA.swapaxes(1,2), BA.swapaxes(1,2)])/2
Rule0 += np.einsum("Iij -> I", H2[1, 1, BB, BB, BB.swapaxes(1,2), BB.swapaxes(1,2)])/2

H_CI = np.diag(Rule0)

Slater-Condon Rule 1

In [9]:
##  indices for 1-difference
I_A, J_A = np.where( np.all(SpinDifference==np.array([1,0], dtype=CI_dt), axis=2) )
I_B, J_B = np.where( np.all(SpinDifference==np.array([0,1], dtype=CI_dt), axis=2) )
I_A = I_A.astype(CI_dt)
J_A = J_A.astype(CI_dt)
I_B = I_B.astype(CI_dt)
J_B = J_B.astype(CI_dt)

A_t  = (np.where( Difference[I_A, J_A, 0] ==  1 )[1]).astype(CI_dt)
A    = (np.where( Difference[I_A, J_A, 0] == -1 )[1]).astype(CI_dt)
B_t  = (np.where( Difference[I_B, J_B, 1] ==  1 )[1]).astype(CI_dt)
B    = (np.where( Difference[I_B, J_B, 1] == -1 )[1]).astype(CI_dt)
CA_i = ((np.where( Sum[I_A, J_A, 0] == 2 )[1]).reshape(len(I_A), N_s[0]-1)).astype(CI_dt)
CB_i = ((np.where( Sum[I_B, J_B, 1] == 2 )[1]).reshape(len(I_B), N_s[1]-1)).astype(CI_dt)

if I_A.size > 0:
  H_CI[I_A, J_A]  = H1[0, A, A_t]
  H_CI[I_A, J_A] += np.einsum("In -> I", H2[0, 0, Ext(A, N_s[0]-1), Ext(A_t, N_s[0]-1), CA_i, CA_i])
  H_CI[I_A, J_A] += np.einsum("Ii -> I", H2[0, 1, Ext(A, N_s[1]  ), Ext(A_t, N_s[1]  ), ΛB[I_A], ΛB[J_A]])
  H_CI[I_A, J_A] *= Γ_Isp[I_A, 0, A_t] * Γ_Isp[J_A, 0, A]

if I_B.size > 0:
  H_CI[I_B, J_B]  = H1[1, B, B_t]
  H_CI[I_B, J_B] += np.einsum("In -> I", H2[1, 1, Ext(B, N_s[1]-1), Ext(B_t, N_s[1]-1), CB_i, CB_i])
  H_CI[I_B, J_B] += np.einsum("Ii -> I", H2[1, 0, Ext(B, N_s[0]  ), Ext(B_t, N_s[0]  ), ΛA[I_B], ΛA[J_B]])
  H_CI[I_B, J_B] *= Γ_Isp[I_B, 1, B_t] * Γ_Isp[J_B, 1, B]

Slater-Condon Rule 2

In [10]:
## indices for 2-differences
I_AA, J_AA = np.where( np.all(SpinDifference==np.array([2,0], dtype=CI_dt), axis=2) )
I_BB, J_BB = np.where( np.all(SpinDifference==np.array([0,2], dtype=CI_dt), axis=2) )
I_AB, J_AB = np.where( np.all(SpinDifference==np.array([1,1], dtype=CI_dt), axis=2) )
I_AA = I_AA.astype(CI_dt)
J_AA = J_AA.astype(CI_dt)
I_BB = I_BB.astype(CI_dt)
J_BB = J_BB.astype(CI_dt)
I_AB = I_AB.astype(CI_dt)
J_AB = J_AB.astype(CI_dt)

AA   = (np.where( Difference[I_AA, J_AA, 0] == -1)[1].reshape(len(I_AA),2).T).astype(CI_dt)
AA_t = (np.where( Difference[I_AA, J_AA, 0] ==  1)[1].reshape(len(I_AA),2).T).astype(CI_dt)
BB   = (np.where( Difference[I_BB, J_BB, 1] == -1)[1].reshape(len(I_BB),2).T).astype(CI_dt)
BB_t = (np.where( Difference[I_BB, J_BB, 1] ==  1)[1].reshape(len(I_BB),2).T).astype(CI_dt)
AB   = np.asarray([ np.where( Difference[I_AB, J_AB, 0] == -1 )[1], np.where( Difference[I_AB, J_AB, 1] == -1 )[1] ], dtype=CI_dt)
AB_t = np.asarray([ np.where( Difference[I_AB, J_AB, 0] ==  1 )[1], np.where( Difference[I_AB, J_AB, 1] ==  1 )[1] ], dtype=CI_dt)

if I_AA.size > 0: 
    H_CI[I_AA, J_AA]  = H2[0, 0, AA[0], AA_t[0], AA[1], AA_t[1]]
    H_CI[I_AA, J_AA] *= Γ_Isp[I_AA, 0, AA[0]] * Γ_Isp[J_AA, 0, AA_t[0]] * Γ_Isp[I_AA, 0, AA[1]] * Γ_Isp[J_AA, 0, AA_t[1]]
if I_BB.size > 0: 
    H_CI[I_BB, J_BB]  = H2[1, 1, BB[0], BB_t[0], BB[1], BB_t[1]]
    H_CI[I_BB, J_BB] *= Γ_Isp[I_BB, 1, BB[0]] * Γ_Isp[J_BB, 1, BB_t[0]] * Γ_Isp[I_BB, 1, BB[1]] * Γ_Isp[J_BB, 1, BB_t[1]]
if I_AB.size > 0:
    H_CI[I_AB, J_AB]  = H2[0, 1, AB[0], AB_t[0], AB[1], AB_t[1]]
    H_CI[I_AB, J_AB] *= Γ_Isp[I_AB, 0, AB[0]] * Γ_Isp[J_AB, 0, AB_t[0]] * Γ_Isp[I_AB, 1, AB[1]] * Γ_Isp[J_AB, 1, AB_t[1]]

Comparison

In [11]:
cisolver = fci.FCI(uhf,singlet=False)
cisolver.nroots = 100 # 100
cisolver.spin = 0
cisolver.davidson_only = False
pyscf_fci_energy = cisolver.kernel()[0]

nuclear_rep = uhf.energy_nuc()
e_fci_my, X_IJ = np.linalg.eigh(H_CI)

print("pyscf : " + str(pyscf_fci_energy[0] ))
print("Mine  : " + str( e_fci_my[0] + nuclear_rep ))

pyscf : -59.68748824964669
Mine  : -59.68748824964675
